In [1]:
%matplotlib notebook
import numpy as np
from sympy import *
#from sympy.abc import a,b
init_printing()
import math
from sympy.printing.theanocode import theano_function
from matplotlib import pyplot as plt
from pyphysim.util.misc import randn_c


In [3]:
def lms(w0, u, d, s):
    """
    Find the LMS filter coefficients.

    Parameters
    ----------
    w0 : np.ndarray
        Init weight vector with M taps. Dimension: (M,)
    u : np.ndarray
        Input data vector. Dimension: (N,)
    d : Desired Output data vector. Dimension: (N,)
    s : Step size.
    """
    N = u.size
    M = w0.size
    w = w0
    x = np.zeros_like(w0)
    for k in range(N):
        x = np.hstack([u[k], x[:-1]])
        ek = d[k] - w.conj() @ x
        dw = s * ek.conj() * x
        w = w + dw

    return w


In [24]:
pilots = np.array(
    [-1, -1, -1, +1, +1, -1, -1, -1, +1, +1, +1, +1, -1, +1, +1, -1, +1, -1])
N = pilots.size
M = 6  # Number of filter coefficients
Lh = 2  # Channel memory
# Lp = pilots.size  # Number of trainning symbols
LD = 150  # Number of data symbols
LG = Lh + M  # Number of transmitted zeroes (guard interval)

# Canal
h = np.array([4, -5])  # A channel with 2 taps

w0 = np.zeros(M)
d = pilots
step = 0.0015



# Transmit pilots through the channel
u = np.convolve(pilots, h)[:N]  # Don't include elements after we finished transmission

# Fild filter coefficients using LMS algorithm
w = lms(w0, u, pilots, step)

#tilde_pilots = np.convolve(w.conj(),u)



data_points = 1-2*np.random.randint(0,2,LD)
guard_interval = np.zeros(LG)

In [33]:
# Transmit one frame
frame_data = np.hstack([pilots, data_points, guard_interval])


out = 0
for i in range(N):
    


SyntaxError: invalid syntax (<ipython-input-33-b578247acca8>, line 4)